# Loft Surface Creation

This notebook focuses on the creation of a loft surface. 
A loft surface is a type of NURBS (Non-Uniform Rational B-Spline) surface that is generated by passing through a sequence of compatible NURBS curves. 
Compatibility in this context means that the curves have the same degree and share the same flat knots. 

The following sections will demonstrate the mathematical proof and application of this concept.


## Mathematical Proof of Loft Surface Creation

A loft surface, denoted as $S(u, v)$, is a NURBS surface that intersects a set of NURBS curves $\{ C_j \}$ at specific parameter values. The curves in the set are compatible, meaning they have the same degree and flat knots. 

Given a set of curves $\{ C_j \}$, each curve can be represented as:
$$
C_j(u) = \sum_{i} N_{i, p}(u) P_i^{(j)}
$$
And the loft surface $S(u, v)$ is defined as:
$$
S(u, v) = \sum_{i} \sum_{j} N_{i, p}(u) N_{j, q}(v) P_{i, j}
$$

By definition, for a given parameter $v_{j0}$, the surface $S$ is coincident with the curve $C_j$:
$$
C_j(u) = S(u, v_j0) = \sum_{i} N_{i, p}(u) \sum_{j} N_{j, q}(v_{j0}) P_{i, j}
$$

Through identification, we establish the relationship:
$$
P_i^{(j0)} = \sum_{j} N_{j, q}(v_{j0}) P_{i, j}
$$

This leads to a linear system equation for each value of $i$, facilitating the calculation of surface points.


## Solving the Linear System

For each value of $i$, the relationship established above leads to a linear system represented by the following matrix equation:
$$
\begin{pmatrix}
N_{0,q}(v_0) & \cdots & N_{i,q}(v_0) & \cdots & N_{m,q}(v_0)\\ 
\vdots & \ddots & \vdots & \ddots & \vdots\\ 
N_{0,q}(v_{j0}) & \cdots & N_{j,q}(v_{j0}) & \cdots & N_{m,q}(v_{j0})\\ 
\vdots & \ddots & \vdots & \ddots & \vdots\\ 
N_{0,q}(v_m) & \cdots & N_{i,q}(v_m) & \cdots & N_{m,q}(v_m)
\end{pmatrix}
\begin{pmatrix}
P_{i,0}\\ 
\vdots\\ 
P_{i,j}\\ 
\vdots\\ 
P_{i,m}
\end{pmatrix} = 
\begin{pmatrix}
P_{i}^{(0)}\\ 
\vdots\\ 
P_{i}^{(j)}\\ 
\vdots\\ 
P_{i}^{(m)}
\end{pmatrix}
$$

This matrix equation is critical as it requires only a single matrix inversion to solve for the surface poles, significantly simplifying the computational process.


## Application Example: Defining Curves

In this section, we define three different NURBS curves, each with a unique set of control points (poles). These curves will be used to create the loft surface.


In [ ]:
from pygbs import gbs

# Define the control points (poles) for three different curves
poles1 = [
    [0.0, 0.0, 0.0],
    [0.4, 0.4, 0.0],
    [0.6, 0.1, 0.0],
    [1.0, 0.0, 0.0],
]

poles2 = [
    [0.0, 0.0, 0.5],
    [0.4, 0.1, 0.5],
    [0.6, 0.4, 0.5],
    [1.0, 0.0, 0.5],
]

poles3 = [
    [0.0, 0.0, 1.0],
    [0.4, 0.3, 1.0],
    [0.6, 0.3, 1.0],
    [1.0, 0.0, 1.0],
]

# Defining the multiplicities and the degree of the curves
mults = [4, 4]
p = 3
k = [0., 1.]

# Creating NURBS curves using the defined control points and parameters
crv1 = gbs.BSCurve3d(poles1, k, mults, p)
crv2 = gbs.BSCurve3d(poles2, k, mults, p)
crv3 = gbs.BSCurve3d(poles3, k, mults, p)


### Plotting the Defined Curves

After defining the curves, we use PyVista and a custom plotting function to visualize them. This step is important to verify the shapes and positions of the curves before proceeding to loft surface creation.


In [ ]:
import pyvista as pv
from pygbs import vistaplot as gbv

# Initialize the PyVista plotter
plotter = pv.Plotter()

# Add the defined curves to the plotter for visualization
gbv.add_curves_to_plotter([crv1, crv2, crv3], plotter)

# Display the plot with the curves
plotter.show(jupyter_backend='panel')


### Building the Surface Parametrization

Now, we will start constructing the surface parametrization in the $v$ direction. This involves defining the number of curves, the degree of the surface in the $v$ direction, and the flat knots for both $u$ and $v$ parameters.


In [ ]:
import numpy as np

# Define the number of curves and the number of control points in the u-direction
n_crv = 3
nu = 4
flat_u = gbs.flat_knots(k, mults)  # Generate flat knots for u-direction

# Define the degree in the v-direction and the parameter values in v
q = 2
v = [0., 0.5, 1.]
flat_v = gbs.build_simple_mult_flat_knots(v, q)  # Generate flat knots for v-direction


### Preparing Curve Poles for Surface Construction

The control points of the curves are appended and then transposed. This step is crucial for isolating the poles at a constant index $i$, which is necessary for the upcoming matrix operations in the surface construction process.


In [ ]:
# Append and transpose the curve poles for further processing
poles_crvs = np.array([poles1, poles2, poles3])
poles_crvs_t = np.transpose(poles_crvs, (1, 0, 2))


### Building and Solving the Basis Function Matrix

In this step, we build the basis function matrix for the surface construction. This matrix is then inverted to solve for the surface control points. The inversion of this matrix is a key step in determining the surface shape.


In [ ]:
# Build the basis function matrix for the v-direction
M = np.array([[gbs.basis_function(v_, j, q, 0, flat_v) for j in range(n_crv)] for v_ in v])

# Invert the matrix to solve for surface control points
M_inv = np.linalg.inv(M)


### Constructing the Surface Poles

After solving the basis function matrix, we now construct the poles of the surface. This step involves matrix multiplication to calculate the control points of the surface.


In [ ]:
# Calculate the control points for the surface using matrix multiplication
poles_srf_t = np.array([np.dot(M_inv, poles_crvs_t[i]) for i in range(nu)])


### Reorienting Surface Poles

Before creating the surface, it is necessary to reorient the surface poles to the correct $uv$ orientation. This involves transposing and reshaping the array of surface poles.


In [ ]:
poles_srf = np.transpose(poles_srf_t, (1, 0, 2))
# Flattent poles
poles_srf = poles_srf.reshape((-1, 3))

Then we finally create the surface and display the result.

In [ ]:
srf = gbs.BSSurface3d(poles_srf.tolist(), flat_u, flat_v, p, q)
gbv.add_surfaces_to_plotter([srf], plotter)
plotter.show(jupyter_backend='panel')

### Checking the mid curve and the surface are coincident on the specified parameter.

In [ ]:
from pytest import approx
for u in np.linspace(0., 1., 15):
    assert crv2(u) == approx(srf(u,0.5))